### Melakukan import library yang dibutuhkan

In [1]:
import pandas as pd
import requests
import bs4
import time
from fake_useragent import UserAgent
import math
import re

### Membaca dataset training dan testing

In [2]:
df = pd.read_csv("isaPerson.csv")

In [3]:
df.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,followers_count,friends_count,geo_enabled,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,url,verified,username,isaPerson
0,False,08/02/2015 0:32,True,False,NaN,NaN,1,16858,6,False,...,DDEEF6,333333,True,False,otodriver,17695,NaN,False,@otodriver,nonperson
1,False,07/07/2009 13:13,False,False,Markas Pusat Palang Merah Indonesia | Jalan Je...,NaN,2247,1033888,1235,True,...,F5BACB,333333,True,False,palangmerah,53538,https://t.co/mx3e3UYXPi,True,@palangmerah,nonperson
2,False,16/11/2011 10:36,False,False,Ada diskon setiap hari cek di aplikasi #Pegipe...,NaN,1439,46217,36,True,...,FFFFFF,333333,True,False,pegipegi,35329,https://t.co/lP16dITceB,True,@pegipegi,nonperson
3,False,13/04/2008 03:30,False,False,#Ngahiji Menang Bersama! Download #PERSIBApp ð...,NaN,1464,3502961,90,True,...,FFF7CC,0C3E53,True,False,persib,46556,https://t.co/hqHLwIrxfY,True,@persib,nonperson
4,False,19/09/2014 03:05,False,False,Akun Resmi | Dikelola Subbaghumas | Call Cente...,NaN,316,26707,204,True,...,0,0,False,False,polresjogja,9204,https://t.co/XrZB8YcypN,True,@polresjogja,nonperson


In [4]:
td = pd.read_csv('tdsd.tsv', sep="\t")

In [5]:
td.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,entities,favourites_count,followers_count,friends_count,geo_enabled,...,protected,screen_name,statuses_count,url,utc_offset,verified,username,isaPerson,userLocation,jobArea
0,False,16/01/2010 03:52,False,False,Industrial Engineering of Pancasila University...,NaN,153.0,146.0,257.0,True,...,False,BobidDams,6993.0,NaN,NaN,False,@BobidDams,1.0,NaN,NaN
1,False,01/12/2009 2:33,False,False,Official account of Grand Indonesia a one-stop...,NaN,1957.0,765907.0,348.0,True,...,False,GrandIndo,32272.0,https://t.co/qqFyZ8MBI3,NaN,True,@GrandIndo,0.0,NaN,NaN
2,False,16/04/2015 03:53,True,False,Official account of PERSIS Solo. #LaskarSamber...,NaN,22.0,34867.0,22.0,True,...,False,persisofficial,5285.0,https://t.co/I43yb1p3Pm,NaN,False,@persisofficial,1.0,NaN,NaN
3,False,02/10/2010 1:57,False,False,i am you,NaN,3825.0,310.0,1130.0,True,...,False,hamsil_,11815.0,NaN,NaN,False,@hamsil_,1.0,NaN,NaN
4,False,21/08/2010 11:05,False,False,NaN,NaN,7.0,347.0,316.0,True,...,False,chrisryantonius,29730.0,NaN,NaN,False,@chrisryantonius,1.0,NaN,NaN


### Melakukan preprocessing data dengan mengubah data yang kosong menjadi ""

In [6]:
td['profile_location'] = td['profile_location'].fillna('')

In [7]:
td['profile_location'].unique()

array(['Jakarta Capital Region', 'Jakarta, Indonesia',
       'Surakarta, Jawa Tengah', ..., 'Jl.HR.Rasuna Said Kav.C-19 Jkt',
       'Medan Petisah, Indonesia', "women's bra"], dtype=object)

In [8]:
td['description'] = td['description'].fillna('')

### Membersihkan data pada kolom description dan location agar bersih dari simbol dan hanya tersisa teks 

In [9]:
import requests
import bs4
from random import randint

selected_sd = {'username':'', 'location':'', 'description': ''}

collection = pd.DataFrame(index=td.index.copy(), columns=selected_sd)

for i in range(0,4153):
    desc = ""
    loca = ""
    if(td.loc[i, 'description'] != ''):
        desc = td.loc[i, "description"].lower()
    if(pd.isna(td.loc[i, 'profile_location']) != True):
        loca = str(td.loc[i, "profile_location"]).lower()
    
    selected_sd['username'] = td.loc[i, "username"]
    selected_sd['location'] = re.sub(r'[^\w]', ' ', loca)
    selected_sd['description'] = re.sub(r'[^\w]', ' ', desc)
    selected_sd['location'] = re.sub(r'[!@#$]', ' ', loca)
    selected_sd['description'] = re.sub(r'[!@#$]', ' ', desc)
    loca = loca.replace(",", ' ')
    loca = loca.replace(".", ' ')
    loca = loca.replace("/", ' ')
    loca = loca.replace("|", ' ')
    loca = loca.replace("&", ' ')
    loca = loca.replace("_", ' ')
    loca = loca.replace("(", ' ')
    loca = loca.replace(")", ' ')
    selected_sd['location'] = loca.replace("-", ' ')
    desc = desc.replace(",", ' ')
    desc = desc.replace(".", ' ')
    desc = desc.replace("/", ' ')
    desc = desc.replace("|", ' ')
    desc = desc.replace("&", ' ')
    desc = desc.replace("_", ' ')
    desc = desc.replace("(", ' ')
    desc = desc.replace(")", ' ')
    selected_sd['description'] = desc.replace(",", ' ')
    print(selected_sd['location'])
    print(selected_sd['description'])
    print("=========")
    collection.loc[i] = pd.Series(selected_sd)

jakarta capital region
industrial engineering of pancasila university   industrial management of bogor agricultural university   wilmar chemical indonesia   #viscabarã§a
jakarta  indonesia
official account of grand indonesia a one-stop shopping for everyone  ph: +62  21  23580001 2
surakarta  jawa tengah
official account of persis solo  #laskarsambernyawa

i am you


bali  indonesia
@ponyourtone â€” booking inquiries : +62 878-8216-7715

manusia biasa yang sedang bertumbuh dan berproses 
indonesia
fasilkom ui 2014
kota surabaya  jawa timur
official account institut teknologi sepuluh nopember  mohon support dan bantuannya akun sedang dalam pemulihan sobat kampus! 
jakarta
be grateful 
depok  indonesia
ctupa7@gmail com
bandung  jawa barat

banda aceh
do what u like
somewhere over the rainbow : 
reverse psychology is the ultimate sarcasm
jakarta
konsultan remaja terbaik!!!!

akun resmi dpd partai demokrat daerah istimewa yogyakarta ðÿ‡®ðÿ‡©
kompas maritim
kalau hidup penuh pilihan  sekara

indonesia
universitas 17 agustus 1945 samarinda  teknik sipil 2013


kompas maritim  smansa



bogor  jawa barat
âœ¨ indonesian millenial mom   lifestyle blogger  ðÿž“ agriculture   ðÿ’œ bts army - try to find my fan acc ðÿ˜†   ðÿ’» #faradiladputridotcom
bekasi  jawa barat
20:23 â€¢ saya menulis agar kepala saya tidak meledak â€” buku terbaru: ð—ð—®ð—»ð—´ð—®ð—» ð—¦ð—²ð—ºð—¯ð˜‚ð—»ð˜†ð—¶ ð—±ð—¶ ð—•ð˜‚ð—ºð—¶ â€¢
bogor
kompas maritim smansa
indonesia
the official husein alatas cp : 087881174433  follow my ig : @husein al atas

official twitter of david @noah id
indonesia
we do everything for one thing  sheer driving pleasure 
unpad  jatinangor
music and education will be your friends!   ðÿ“© radiomu unpad@gmail com   ðÿ“¸ https:  t co 7wjetgbxhh   ðÿ†” lineat: @radiomu



penulis pengurus komunitas salihara jalan salihara 16
indonesia
98'   a servant of god   ig : christynatalie2

di  yogyakarta    ig : @grestyan

jenderal tni  purn  h  djoko santoso
semarang
undip is 

indonesia
official twitter for cbn one of indonesia's leading digital service provider  dsp  since 1996   cbn customer care

the expert in everything was once a beginner

hi!

apprentice python programmer :  https:  t co tvfux8nqkw
yogyakarta  id



jakarta capital region
official twitter badan nasional penanggulangan terorisme   national counter terrorism agency   ig: @bnptri  https:  t co zkedn4qmnm
pengaduan
sub @zoofess id   menfess bot for animal lovers   managed by @kamutuh 



metaphorically standing on a crossroad  a silent reader with occasional temporary outbursts 
sky
pt  triputra negara   m2 residence group   law student
jakarta pusat  dki jakarta
official twitter kementerian sekretariat negara ri  ministry of state secretariat of the republic of indonesia  021-3516952 email: humas at setneg   go   id
universitas sriwijaya
cooking food hunter culinary foodpreanure sharing demo cooking   only unsri cooking club   contact : +6285769475148 email dapur unsri@gmail com â–ª
bandu

üt:  6 209339 106 818756
part of grid network - official account of https:  t co vnysy4eytq media  fun educative   empowering girls in indonesia ðÿ’ðÿ» ðÿ“² 021 533 0150 70 ext 33632
jakarta
ðÿ¢ kampus 1 semanggi ðÿ« kampus 2 pluit ðÿ¬ kampus 3 bsd   telp: 021-5703306 ext 123 434 544   wa: 0813-8181-2800
jakarta   indonesia
peace love 'n respect! â€¢ @armandmaulana @dewabudjana @moththomas @gustihendy â€¢ @amityasia cp : 0811-959-6161 - ig: @gigibandofficial


jakarta pusat  dki jakarta
hidup seperti tokai 


indonesia
pspbi ur'14   just supports each other anâ™¥
jakarta
actor tv host a husband a father ceo dcp shoes industry business owner march shoes and apparel ceo pt archa jaya perkasa 
jakarta capital region  indonesia
memes retweeter
indonesia
23   isfj   eng id   its'15
earth
just a normal person

fanbase mas menteri @nadiemmakarim ðÿ‡®ðÿ‡© mendikbud riâ£ ðÿ‘¨â€ðÿ« berbagi info pendidikan terkini #merdekabelajar ðÿ“© kirim fan art informasi via dm mention
jakarta
a morni

jakarta
co-founder @gojekindonesia
depok  indonesia
hai semua ini akun twitter asli punya oka dan oki!   follow akun instaram kita juga ya di @okk ui   #ragamgelora
wherever i am
a mere peasant

hold   keep learning
jl  gatot subroto no 6 jakarta
akun twitter resmi majelis permusyawaratan rakyat republik indonesia

50% emo tional  50% meme

i am the mentalist   
jakarta
official account   part of @bankmandiri @mandirifiesta @mandiricard @wrausahamandiri   fanpage facebook mandiri care
jakarta capital region
producer @tobaliputraproductions
jakarta indonesia
contact person please call my manager: +62â 821â€‘2267â€‘4777- +62â 818â€‘0802â€‘2422

#name?
jakarta capital region
andien management: +62 8118442706   info@andienaisyah com
g ab
if you wanna do right thing let's do it right way #lfc #ynwa

akun twitter resmi mongabay indonesia yg berfokus pd hutan juga menyediakan berita analisis   informasi lain yg berhubungan dengan lingkungan 
sumatera utara  indonesia
gubernur sumatera utara
d

...

jakarta  indonesia
--
jakarta pusat indonesia
masa lampau selalu aktual  media sejarah populer pertama di indonesia  member of gvm networks  kontak: layanan@historia id 
life school
an educationist an it practitioner and a husband the most beautiful wife :  and now a father of the most cute hansom sons 
indonesia

jakarta  indonesia  
duniaku #bumimanusia dengan segala persoalannya  minke   so let's make the next big things !!
@kontakindomaret
subscribe channel youtube kami di https:  t co povmvpuwnh
üt:  3 334706 114 604063
maap  lumayan jarang nge twit  
dki jakarta   banten
ðÿž¬ content creator: review travel-food storytelling skin   hair treatment on youtube   ðÿ‘¨â€ðÿ’»social media enthusiast   blog: https:  t co enh2g0hs0m
tangerang
@in8inite    @wetquatre    @angklung2013    @cgtfasilkom    @blueredstudio    @comdevfasilkom


jakarta
jakarta javascript user group  we meet every month to discuss or hacks interesting stuffs about javascript  #jakartajs
deep down in your tanjidor


üt:  6 227994 106 827188
music promoter known as java musikindo
jakarta  indonesia
#1newstainment follow â¬‡ï¸â¬‡ï¸â¬‡ï¸ @vivanewscom   @100kpjcom   @intipseleb   @vlixid   @jagodangdutcom   @sahijabcom
indonesia
indonesia welcome to vice 

singer songwriter   booking contact: contact@ifyalyssa com   selamat mendengarkan album â€œpelita laraâ€âœ¨

path to paradise begins in hell
dki jakarta  indonesia
join us to spread love happiness and kindness    https:  t co k8nfohiofv    @inakafstore    @in tertainment
rancaekek kencana   bandung 
be the good because god loves the goodness feb '17

a collection of paradoxes
bogor  indonesia
haha hihi adalah kunci
singgah sementara di bintaro
egois pesimis tdk realistis sinis dan apatis urusan kerja hub abeng 08562137056
indonesia
official account of universitas terbuka  ut  indonesia open university   perguruan tinggi negeri dgn sistem pendidikan terbuka dan jarak jauh   hallo ut 1500024
adolescentfams
this is how we do!
jakarta capital region



banjarmasin  indonesia
manusia sama tp mempunyai nasib yg beda-beda 22 juni 1995 fkip matematika unlam smasix bjm  spenda bjm  sdn teluk dalam 9 bjm tk islam mesjid jami 
balikpapan  indonesia
cold little heart
indonesia
penulis buku #bincangakhlak  2019  #sobatsakit  2020 â–ªï¸business: 081222147512 - barlie ve@gmail com @barlieve
asiu!!!
â†» ðð€ð‘ðžðƒð˜ â•± 1992 â» exo's member  he is a handsome boy and so cool byun baekhyun is the name
jakarta capital region  indone

dki jakarta  indonesia
share pengalaman gojek di #gojek24jam
jakarta   indonesia
bca senantiasa di sisi anda 24 x 7 telepon 1500888

fashionable kidswear brand  mall kelapa gading 3 level 2 grand indonesia west mall level 2 plaza indonesia level 3 part of time international
üt:  6 24905 106 596544
â€œproduk gagalâ€ dari sekolah belum tentu gagal di kehidupan 
louvres  france
write sins not status 
under the same sky
dreamer boy

embracing journey of life to see what the world has to offer me  or i creat

pamekasan  indonesia
081931515222 - 081357259222 - 083854509222

proud mom    former journalist   psi
jakarta capital region  indone
indonesian transport community  latest news - showcase - development  network @j5throwback @cloud555fm jalur5 shop  contact 08119826800 087855502016
papua
kantor pengawasan dan pelayanan bea dan cukai tipe madya pabean c merauke
indonesia
meneliti dan menerbitkan esai-esai segar yang berisi kajian analisis dan opini seputar media dan komunikasi   donasi: https:  t co noib4pvz1g
indonesia
skinnyindonesian24 from 24 06 2011 until 24 06 2021  nonton dpr musikal  klik link di bawah 
jakarta  indonesia
blackberryâ® adalah penyedia enterprise software dan jasa yang fokus dalam mengamankan dan mengelola iot endpoints  for more insights visit https:  t co curfb8mnhf

hahay

your tooth fairy 

public health - universitas indonesia epidemiologi - agung aji@ui ac id
indonesia 🇮🇩

yogyakarta  indonesia
student at faculty of psychology ahmad dahlan university yk     m

stage film television actress; television director 

orion '14
jakarta

women's bra
introvert banget orangnya

ig : fiahananda
indonesia
the king of my convenience   ain't gonna lie my @kelesan id is the best!!   life either daring adventure or nothing at all ~ helen keller
jakarta  indonesia

jakarta capital region
akun resmi otoritas jasa keuangan  ojk    jl  lapangan banteng timur 2-4  jakarta 10710   kontak ojk : 157  ojk tv : https:  t co vlgh2vswtd

think a thing


dki jakarta  indonesia
jangan di follow cuma buat stalking akun alter aja   

ikhtiar dan takdir
indonesia
jurnalisme data  media analisis  infografik ciamik 
jawa timur  indonesia
ð’œð’†ð’†ð’‘ ð’„ð’‚ð’ð’ž ð’‚ð’ð’… ð’ƒð’†ð’ð’šð’†ð’—ð’† ð’•ð’ ð‘®ð’ð’… ðÿ˜‡


In [10]:
collection.head()

,username,location,description
0,@BobidDams,jakarta capital region,industrial engineering of pancasila university...
1,@GrandIndo,jakarta indonesia,official account of grand indonesia a one-stop...
2,@persisofficial,surakarta jawa tengah,official account of persis solo #laskarsamber...
3,@hamsil_,,i am you
4,@chrisryantonius,,


In [11]:
collection.to_csv("input.csv".format(2), index=False)

### Leksikon yang digunakan untuk mengekstrak lokasi dari akun Twitter

In [12]:
def ekstraksi_lokasi(list_tweet):
    jabodetabek = ["jawa", "ui", "jakarta", "dki jakarta", "bogor", "depok", "tangerang", "bekasi", "kepulauan seribu", "jakarta barat", "jakarta pusat", "jakarta selatan", "jakarta utara", "jakarta timur", "kabupaten administrasi kepulauan seribu", "kabupaten kepulauan seribu", "slipi", "kota administrasi jakarta barat", "cibubur", "kota jakarta barat", "cikarang", "kota administrasi jakarta pusat", "kota jakarta pusat", "kota administrasi jakarta selatan", "kota jakarta selatan", "kota administrasi jakarta timur", "cipayung", "kota jakarta timur", "kota administrasi jakarta utara", "kota jakarta utara", "kota bogor", "serpong", "kota depok", "kota tangerang", "kota bekasi", "jakarta capital region", "dki", "jkt", "bgr", "pasar", "minggu", "batavia", "tangsel", "bintaro", "bsd", "pamulang", "ciputat", "ciledug", "aren", "jaksel", "jaktim", "kuningan", "kalibata", "rawa", "buaya", "cilangkap", "cipayung", "mabes", "cawang", "kemang", "palmerah", "tebet", "rawabadung", "benhil", "sudirman" "ancol", "karet", "jagakarsa", "thamrin", "manggarai", "cibinong", "nambo", "baranangsiang", "gbk", "gambir"]
    sumatera = ["sumatera", "west", "sumatra", "sumsel", "sumbar", "sumut", "padang", "padang panjang", "riau", "jambi", "bengkulu", "sumatera selatan", "kepulauan bangka belitung", "bangka", "lampung", "aceh", "nanggroe aceh darussalam", "banda aceh", "nanggroe aceh darussalam", "nad" "bukittinggi", "bukit tinggi", "pasaman barat", "pasaman", "pariaman", "padangpanjang", "sawahlunto", "solok", "palembang", "nias", "nias barat", "nias selatan", "tapanuli", "toba", "binjai", "kabupaten agam", 
                "kabupaten dharmasraya", "kabupaten kepulauan mentawai", "kabupaten lima puluh kota", "kabupaten padang pariaman", 
                "sriwijaya", "kabupaten pasaman", "kabupaten pasaman barat", "kabupaten pesisir selatan", "kabupaten sijunjung", 
                "kabupaten solok", "kabupaten solok selatan", "kabupaten tanah datar", "kota bukittinggi", "kota padang", 
                "kota padangpanjang", "kota pariaman", "tanjungpinang", "kota payakumbuh", "kota sawahlunto", "kota solok", 
                "agam", "dharmasraya", "kepulauan mentawai", "mentawai", "lima puluh kota", "padang pariaman", "pasaman", 
                "pasaman barat", "pesisir selatan", "sijunjung", "solok", "solok selatan", "tanah datar", "bukittinggi", "padang", 
                "payakumbuh", "sawahlunto", "kabupaten banyuasin", "kabupaten empat lawang", "kabupaten lahat", "kabupaten muara enim", 
                "kabupaten musi banyuasin", "kabupaten musi rawas", "kabupaten musi rawas utara", "kabupaten ogan ilir", 
                "kabupaten ogan komering ilir", "kabupaten ogan komering ulu", "kabupaten ogan komering ulu selatan", 
                "kabupaten ogan komering ulu timur", "kabupaten penukal abab lematang ilir", "kota lubuklinggau", "kota pagar alam", 
                "kota palembang", "kota prabumulih", "lahat", "muara enim", "musi banyuasin", "musi rawas", "musi rawas utara", "ogan ilir", 
                "ogan komering ilir", "ogan komering ulu", "ogan komering ulu selatan", "ogan komering ulu timur", 
                "penukal abab lematang ilir", "lubuklinggau", "pagar alam", "palembang", "pangkalpinang", "prabumulih", "kabupaten asahan", 
                "kabupaten batu bara", "kabupaten dairi", "kabupaten deli serdang", "kabupaten humbang hasundutan", "kabupaten karo", 
                "kabupaten labuhanbatu", "kabupaten labuhanbatu selatan", "kabupaten labuhanbatu utara", "kabupaten langkat", 
                "kabupaten mandailing natal", "kabupaten nias", "kabupaten nias barat", "kabupaten nias selatan", "kabupaten nias utara", 
                "kabupaten padang lawas", "kabupaten padang lawas utara", "kabupaten pakpak bharat", "kabupaten samosir", 
                "kabupaten serdang bedagai", "kabupaten simalungun", "kabupaten tapanuli selatan", "kabupaten tapanuli tengah", 
                "kabupaten tapanuli utara", "kabupaten toba", "kota binjai", "kota gunungsitoli", "kota medan", "kota padangsidimpuan", 
                "kota pematangsiantar", "kota sibolga", "kota tanjungbalai", "kota tebing tinggi", "asahan", "batu bara", "dairi", 
                "deli serdang", "humbang hasundutan", "karo", "labuhanbatu", "labuhanbatu selatan", "labuhanbatu utara", "langkat", 
                "mandailing natal", "nias", "nias barat", "nias selatan", "nias utara", "padang lawas", "padang lawas utara", "pakpak bharat", 
                "samosir", "serdang bedagai", "simalungun", "tapanuli selatan", "tapanuli tengah", "tapanuli utara", "toba", "binjai", 
                "gunungsitoli", "medan", "padang sidempuan", "pematangsiantar", "sibolga", "tanjungbalai", "tebing tinggi", 
                "kabupaten aceh barat", "kabupaten aceh barat daya", "kabupaten aceh besar", "kabupaten aceh jaya", "kabupaten aceh selatan", 
                "kabupaten aceh singkil", "kabupaten aceh tamiang", "kabupaten aceh tengah", "kabupaten aceh tenggara", 
                "kabupaten aceh timur", "kabupaten aceh utara", "kabupaten bener meriah", "jonggol", "kabupaten bireuen", 
                "kabupaten gayo lues", "kabupaten nagan raya", "kabupaten pidie", "kabupaten pidie jaya", "kabupaten simeulue", 
                "kota banda aceh","kota langsa", "kota lhokseumawe", "kota sabang", "kota subulussalam", "aceh barat", "aceh barat daya", 
                "aceh jaya", "aceh selatan", "aceh singkil", "aceh tamiang", "aceh tengah", "aceh tenggara", "aceh timur", "aceh utara", 
                "bener meriah", "bireuen", "gayo", "gayo lues", "nagan raya", "pidie", "pidie jaya", "simeulue", "banda aceh", "langsa", 
                "lhokseumawe", "sabang", "subulussalam", "kabupaten bengkulu selatan", "kabupaten bengkulu tengah", 
                "kabupaten bengkulu utara", "kabupaten kaur", "kabupaten kepahiang", "kabupaten lebong	", "kabupaten mukomuko", 
                "kabupaten rejang lebong", "kabupaten seluma", "kota bengkulu", "bengkulu", "bengkulu selatan", "bengkulu tengah", 
                "bengkulu utara", "kaur", "kepahiang", "lebong", "mukomuko", "rejang lebong", "seluma", "kabupaten bengkalis", 
                "kabupaten indragiri hilir", "kabupaten indragiri hulu", "kabupaten kampar", "kabupaten kepulauan meranti", 
                "kabupaten kuantan singingi", "kabupaten pelalawan", "kabupaten rokan hilir", "kabupaten rokan hulu", "kabupaten siak", 
                "kota dumai", "kota pekanbaru", "bengkalis", "indragiri hilir", "indragiri hulu", "kampar", "kepulauan meranti", "meranti", 
                "kuantan singingi", "pelalawan", "batam", "rokan hilir", "rokan hulu", "siak", "dumai", "pekanbaru", "sungai", "liat" "siantar", "kuala", "tungkal"]
    jawa_timur = ['jawa', "timur", "east", "java", "banyuwangi", "surabaya", "bangkalan", "blitar", "bojonegoro", "bondowoso", "gresik", "jember", "jombang", "kediri", "lamongan", "lumajang", "madiun", "magetan", "malang", "mojokerto", "nganjuk", "ngawi", "pacitan", "pamekasan", "bangil", "ponorogo", "kraksaan", "sidoarjo", "sampang", "tuban", "tulungagung", "pasuruan", "probolinggo", "batu", "situbondo", "jatim", "sumenep"]
    sulawesi  = ["sulawesi", "sulawesi selatan", "makassar", "sulawesi utara", "manado", "kendari", "sulawesi tenggara", "palu", "sulawesi tengah", "gorontalo", "bitung", "sulawesi utara", "palopo", "sulawesi selatan", "baubau", "wajo", "kolaka", "sulawesi tenggara", "parepare", "sulawesi selatan", "kotamobagu", "sulawesi utara", "makasar", "wakatobi", "tomohon"]
    jawa_barat_banten = ["jawa", "west", "java", "banten", "bandung", "itb", "kabupaten bandung", "bandung barat", "kabupaten bandung barat", "kabupaten ciamis", "ciamis", "kabupaten cianjur", "cianjur", "kabupaten cirebon", "cirebon", "kabupaten garut", "garut", "kabupaten indramayu", "indramayu", "kabupaten karawang", "karawang", "kabupaten kuningan", "kuningan", "kabupaten majalengka", "majalengka", "kabupaten pangandaran", "pangandaran", "purwakarta", "kabupaten purwakarta", "kabupaten subang", "subang", "kabupaten sukabumi", "sukabumi", "kabupaten sumedang", "sumedang", "kabupaten tasikmalaya", "tasikmalaya", "jatinangor", "kota bandung", "kota banjar", "kota cimahi", "kota cirebon", "kota sukabumi", "kota tasikmalaya", "jabar", "bdg"]
    jawa_tengah_yogyakarta = ["jawa", "tengah", "central", "java", "yogyakarta", "kaliwungu", "central java", "banjarnegara", "purwokerto", "batang", "blora", "boyolali", "brebes", "cilacap", "demak", "purwodadi", "grobogan", "jepara", "karanganyar", "kebumen", "kendal", "klaten", "kudus", "mungkid", 'YK', "pati", "baturaden", "kajen", "pemalang", "purbalingga", "purworejo", "rembang", "ungaran", "sragen", "sukoharjo", "slawi", "temanggung", "wonogiri", "wonosobo", "pekalongan", "magelang", "salatiga", "semarang", "central java", "surakarta", "tegal", "bantul", "gunungkidul", "kulon progo", "sleman", "diy", "jogja", "jateng", "solo"]
    kalimantan = [ "kalimantan", "kabupaten balangan", "balangan", "kabupaten banjar", "banjar", "samarinda", "barito", "kuala", "barito", "berau", "kabupaten hulu sungai selatan", "kabupaten hulu sungai tengah", "kabupaten hulu sungai utara", "kabupaten kotabaru", "kabupaten tabalong", "kabupaten tanah bumbu", "kabupaten tanah laut", "kabupaten tapin", "kota banjarbaru", "kota banjarmasin", "kotabaru", "tabalong", "tanah bumbu", "tanah laut", "tapin", "banjarbaru", "banjarmasin", "kalimantan barat", "kalimantan tengah", "kalimantan selatan", "pontianak", "tarakan", "kalsel", "kaltim", "kalbar", "kalimantan timur", "kalimantan utara", "bulungan", "balikpapan", "borneo", "singkawang"]
    maluku_papua = ["maluku utara", "maluku", "kota tual", "ambon", "seram", "seram barat", "maluku tenggara", "maluku tengah", "maluku tengah", "moluccas", "kepulauan aru", "halmahera", "kota ternate", "kota tidore kepulauan", "pulau taliabu", "pulau morotai", "kepulauan sula", "asmat", "papua barat", "papua", "biak", "deiyai", "dogiyai", "intan jaya", "jayapura", "jayawijaya", "keerom", "kepulauan yapen", "lanny jaya", "mappi", "mamberamo tengah", "merauke", "mimika", "nabire", "nduga", "paniai", "pegunungan bintang", "puncak jaya", "sarmi", "supiori", "tolikara", " waropen", "yahukimo", "yalimo", "tidore", "fakfak", "raja ampat", "sorong", "manokwari", "wamena", "kaimana", "asmat"]
    bali_nusatenggara = ["kabupaten badung", "kabupaten bangli", "kabupaten buleleng", "kabupaten gianyar", "nusa", "ubud", "kabupaten jembrana", "kabupaten karangasem", "kabupaten klungkung", "kabupaten tabanan", "kota denpasar", "badung", "bangil", "buleleng", "gianyar", "jembrana", "karangasem", "klungkung", "tabanan", "denpasar", "kuta", "nusa dua", "kabupaten alor", "kabupaten belu", "kabupaten ende", "kabupaten flores timur", "kabupaten kupang", "kabupaten lembata", "kabupaten malaka", "kabupaten manggarai", "kabupaten manggarai barat", "kabupaten manggarai timur", "kabupaten nagekeo", "kabupaten ngada", "kabupaten rote ndao", "kabupaten sabu raijua", "kabupaten sikka", "kabupaten sumba barat", "kabupaten sumba barat daya", "kabupaten sumba tengah", "kabupaten sumba timur",  "kabupaten timor tengah selatan", "kabupaten timor tengah utara", "kota kupang",  "alor", "belu", "ende", "flores", "kupang", "lembata", "malaka", "manggarai", "manggarai barat", "manggarai timur", "nagekeo", "ngada", "rote ndao", "sabu raijua", "sikka", "sumba barat", "sumba barat daya", "sumba tengah", "sumba timur", "timor tengah selatan", "timor tengah utara", "bali", "nusa tenggara", "nusa tenggara timur", "nusa tenggara barat", "east nusa tenggara",  "ntt", "ntb"]

    fitur_lokasi_all = []
    for tweet in list_tweet:
        # inisiasi value
        lokasi = ["jabodetabek", "sumatera", "jawa_timur", "sulawesi", "jawa_barat_banten", "jawa_tengah_yogyakarta", 
                 "kalimantan", "maluku_papua", "bali_nusatenggara"]
        value = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        lokasi_value = {}
        for i in range(len(lokasi)):
            lokasi_value[lokasi[i]] = value[i]
        list_kata = tweet.split()
        for k in list_kata:
            if k in jabodetabek:
                lokasi_value["jabodetabek"] += 1
            if k in sumatera:
                lokasi_value["sumatera"] += 1
            if k in jawa_timur:
                lokasi_value["jawa_timur"] += 1
            if k in sulawesi:
                lokasi_value["sulawesi"] += 1
            if k in jawa_barat_banten:
                lokasi_value["jawa_barat_banten"] += 1
            if k in jawa_tengah_yogyakarta:
                lokasi_value["jawa_tengah_yogyakarta"] += 1
            if k in kalimantan:
                lokasi_value["kalimantan"] += 1
            if k in maluku_papua:
                lokasi_value["maluku_papua"] += 1
            if k in bali_nusatenggara:
                lokasi_value["bali_nusatenggara"] += 1 
        fitur_lokasi_perkalimat = list(lokasi_value.values())
        fitur_lokasi_all.append(fitur_lokasi_perkalimat)
        print(fitur_lokasi_all)
        max_value = max(fitur_lokasi_all[0])
        if(max_value != 0):
            result = fitur_lokasi_all[0].index(max(fitur_lokasi_all[0]))
        else:
            result = -1
    return result

### Membaca data testing yang berisi username, lokasi dan deskripsi

In [13]:
nf = pd.read_csv("input.csv")

In [14]:
nf.head()

,username,location,description
0,@BobidDams,jakarta capital region,industrial engineering of pancasila university...
1,@GrandIndo,jakarta indonesia,official account of grand indonesia a one-stop...
2,@persisofficial,surakarta jawa tengah,official account of persis solo #laskarsamber...
3,@hamsil_,NaN,i am you
4,@chrisryantonius,NaN,NaN


In [15]:
nf['description'] = nf["description"].fillna(" ")
nf['location'] = nf["location"].fillna(" ")

In [16]:
nf["location"].head()

0    jakarta capital region
1        jakarta  indonesia
2    surakarta  jawa tengah
3                          
4                          
Name: location, dtype: object

### Melakukan iterasi setiap row dan memanggil function ekstraksi_lokasi

In [17]:
dfresult = {'0':'', '1':''}
nc = pd.DataFrame(index=td.index.copy(), columns=dfresult)
for i in range(0,4153):
    location = nf.loc[i, 'location']
    description = nf.loc[i, 'description']
    result = ekstraksi_lokasi([location])
    if(result == -1):
        result = ekstraksi_lokasi([description])
        if(result == -1):
            result = 0
    if(result == 0):
        text_result = "jabodetabek"
    elif(result == 1):
        text_result = "sumatera"
    elif(result == 2):
        text_result = "jawa timur"
    elif(result == 3):
        text_result = "sulawesi"
    elif(result == 4):
        text_result = "jawa barat dan banten"
    elif(result == 5):
        text_result = "jawa tengah dan yogyakarta"
    elif(result == 6):
        text_result = "kalimantan"
    elif(result == 7):
        text_result = "maluku dan papua"
    elif(result == 8):
        text_result = "bali dan nusa tenggara"

    dfresult['0'] = i
    dfresult['1'] = text_result
    nc.loc[i] = pd.Series(dfresult)

[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 1, 0, 1, 3, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 3, 0, 1, 1, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 1, 0, 2, 1, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 1, 0, 1, 1, 0, 0, 0]]
[[1, 1, 1, 0, 3, 1, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 3, 0, 1, 1, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 1, 0, 2, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 2, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 2, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 2, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 1, 0, 1, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

...

[[3, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[3, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 2, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 1, 0, 1, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 3, 0, 1, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 1, 0, 0, 1, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 1, 0, 2, 1, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 1, 0, 1, 1, 0, 0, 0]]
[[0, 0, 2, 0, 1, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[1, 0, 1, 0, 2, 1, 0, 0, 0]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 1, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[2, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 1, 0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0,

[[3, 0, 1, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 1, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0]]
[[1, 0, 0,

In [18]:
nc.head(10)

,0,1
0,0,jabodetabek
1,1,jabodetabek
2,2,jawa tengah dan yogyakarta
3,3,jabodetabek
4,4,jabodetabek
5,5,bali dan nusa tenggara
6,6,jabodetabek
7,7,jabodetabek
8,8,jawa timur
9,9,jabodetabek


### Mengeluarkan result dalam bentuk csv

In [19]:
nc.to_csv("submission10.csv".format(2), index=False)